In [141]:
import json
import itertools
import numpy as np
from scipy import stats
from bokeh.charts import Histogram, show
from bokeh.plotting import figure, show, vplot, gridplot
from bokeh.io import output_notebook
from scipy.stats import norm
from bokeh.models import LinearAxis, Range1d, BoxAnnotation, GridPlot

In [142]:
class Data(object):  # each data object contains a time, distance, and rssi value
    
    def __init__(self):
        self.mean = 0
        self.stddev = 0

In [143]:
all_beacons = {}
beacon_ms = {}  # [beaconNum][array with mu/sigmas for each block]

In [144]:
def plot_block():  # currently plots pdf for beacon 0:1 within certain block
    global all_beacons
    global beacon_ms
    all_dist = []
    
    for curr_beacon in all_beacons:
        for i in range(len(all_beacons[curr_beacon])):  # collect all distances for that beacon/block
            all_dist.append(all_beacons[curr_beacon][i])
        #mu, sigma = norm.fit(all_dist)  # get mean and std dev for that beacon/block
        beacon_ms.setdefault(curr_beacon, []).append(Data())  # each beacon has an array of Data
        pos = len(beacon_ms[curr_beacon]) - 1  # position in array
        beacon_ms[curr_beacon][pos].mean = np.mean(all_dist)
        beacon_ms[curr_beacon][pos].stddev = np.std(all_dist)
        all_dist.clear()  # clear array for next beacon

In [145]:
def get_data():
    global all_beacons
    begin = False
    block_num = 0
    beacon_num = "0:0"
    
    f = open("2_6_2016_1441.txt")  # open text file
    for line in f:
        words = line.split()
        parsed_json = json.loads(words[1])  # parse json
        if parsed_json.get('marker'):  # check if marker
            if (parsed_json['marker'] == 'begin\n'):
                begin = True  # begin block
                block_num += 1  # increment block number we are on
            elif (parsed_json.get('marker') == 'end\n'):
                old_begin = begin
                begin = False  # end block
                if (old_begin == True):  # ensures that we have reached the first end after a begin
                    plot_block()  # plot mean vs stddev for that block
                all_beacons.clear()  # resets all_beacons dictionary for next block
        elif parsed_json.get('node') and begin == True:  # check if node and if in block
            beacon_num = str(parsed_json['major']) + ":" + str(parsed_json['minor'])  # beacon identified by major and minor
            rssi = parsed_json['rssi']
            distance = 10**((rssi+60)/-20)  # log-distance pathloss model
            all_beacons.setdefault(beacon_num, []).append(distance)  # each beacon has an array with distances
    f.close()  # close file

In [146]:
def main():
    global beacon_ms
    plots = []
    
    get_data()
    for curr_beacon in beacon_ms:
        p = figure(
            title="Beacon " + str(curr_beacon),
            tools="pan, box_zoom, wheel_zoom, reset, resize",
            x_axis_label='Mean (meters)', y_axis_label='Std Dev',
            plot_width=450, plot_height=225
        )
        for i in range(len(beacon_ms[curr_beacon])):
            p.scatter(
                beacon_ms[curr_beacon][i].mean, beacon_ms[curr_beacon][i].stddev,
                marker="circle", fill_color="firebrick", line_color = "firebrick",
            )
            p.x_range = Range1d(0, 60)
            p.y_range = Range1d(0, 20)
            p.xaxis.axis_label_text_font_size = "12pt"
            p.yaxis.axis_label_text_font_size = "12pt"
        plots.append(p)
        
    output_notebook()
    grid = gridplot([plots[0:2], plots[2:4], plots[4:6], plots[6:]])
    show(grid)

In [147]:
if __name__ == "__main__":
    main()

Loading BokehJS ...